In [ ]:
#!pip install matplotlib psutil tqdm

import torch
import platform
import psutil
import time
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

report_lines = []
report_lines.append(f"=== SYSTEM & GPU REPORT ===")
report_lines.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
report_lines.append("")

# --- System Info ---
report_lines.append("### SYSTEM INFO ###")
os_info = f"{platform.system()} {platform.release()} ({platform.version()})"
cpu_name = platform.processor()
physical_cores = psutil.cpu_count(logical=False)
logical_cores = psutil.cpu_count(logical=True)
ram_gb = round(psutil.virtual_memory().total / 1024**3, 2)

report_lines.append(f"OS: {os_info}")
report_lines.append(f"Processor: {cpu_name}")
report_lines.append(f"CPU Cores: {physical_cores} physical / {logical_cores} logical")
report_lines.append(f"RAM: {ram_gb} GB")
report_lines.append("")

# --- GPU Info ---
if torch.cuda.is_available():
    report_lines.append("### GPU INFO (CUDA) ###")
    n_gpus = torch.cuda.device_count()
    report_lines.append(f"Number of GPUs: {n_gpus}")

    for i in range(n_gpus):
        prop = torch.cuda.get_device_properties(i)
        gpu_name = prop.name
        gpu_vram = prop.total_memory / 1024**3
        report_lines.append(f"\n--- GPU {i} ---")
        report_lines.append(f"Name: {gpu_name}")
        report_lines.append(f"Total VRAM: {gpu_vram:.2f} GB")
        report_lines.append(f"Compute Capability: {prop.major}.{prop.minor}")
        report_lines.append(f"CUDA Cores (estimate): {prop.multi_processor_count * 64}")
        clock_rate = getattr(prop, "clock_rate", None)
        report_lines.append(f"Clock Rate: {clock_rate / 1e3:.0f} MHz" if clock_rate else "Clock Rate: Not available")
        report_lines.append(f"CUDA Version (torch): {torch.version.cuda}")
else:
    report_lines.append("### GPU INFO ###")
    report_lines.append("No CUDA-capable GPU detected.")
    gpu_name = "None"
    gpu_vram = 0
report_lines.append("")

# --- Benchmark ---
report_lines.append("### PERFORMANCE BENCHMARK (Matrix Multiply) ###")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device Selected: ", device)
sizes = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
times = []

try:
    for size in tqdm(sizes, desc="Benchmarking Matrix Multiply"):
        a = torch.randn((size, size), device=device)
        b = torch.randn((size, size), device=device)
        torch.cuda.synchronize() if device.type == 'cuda' else None
        t0 = time.time()
        _ = torch.matmul(a, b)
        torch.cuda.synchronize() if device.type == 'cuda' else None
        t1 = time.time()
        elapsed = t1 - t0
        times.append(elapsed)
        report_lines.append(f"{size}x{size} multiply time: {elapsed:.4f} sec")
        del a, b
        torch.cuda.empty_cache()
except Exception as e:
    report_lines.append(f"Benchmark failed: {e}")

total_time = sum(times)
report_lines.append("")
report_lines.append(f"🔁 Total Matrix Multiply Time: {total_time:.4f} seconds")

# --- Compute Overall Score ---
cpu_score = min(25, physical_cores * 1.0)
ram_score = min(20, ram_gb * 0.25)
gpu_score = min(25, gpu_vram * 1.0)

if total_time <= 1.0:
    benchmark_score = 30
elif total_time <= 1.5:
    benchmark_score = 25
elif total_time <= 2.0:
    benchmark_score = 20
elif total_time <= 3.0:
    benchmark_score = 15
elif total_time <= 5.0:
    benchmark_score = 10
else:
    benchmark_score = 5

total_score = round(cpu_score + ram_score + gpu_score + benchmark_score, 2)

# --- Assign Tier Marker ---
if total_score >= 90:
    tier = "🚀 Extreme Workstation – Top 1% elite performance"
elif total_score >= 80:
    tier = "🔥 Heavy ML/Gaming Machine – Very powerful"
elif total_score >= 65:
    tier = "⚙️ Mid-Range Research Rig – Balanced, capable"
elif total_score >= 50:
    tier = "🧪 Entry-Level ML Machine – Good for prototyping"
elif total_score >= 30:
    tier = "🐢 Basic Productivity System – Limited ML use"
else:
    tier = "❌ Low-Grade System – Not suitable for ML/DL workloads"

# --- Add Score and Tier to Report ---
report_lines.append("")
report_lines.append("🏁 OVERALL SYSTEM PERFORMANCE SCORE")
report_lines.append(f"CPU Score:       {cpu_score:.2f} / 25")
report_lines.append(f"RAM Score:       {ram_score:.2f} / 20")
report_lines.append(f"GPU Score:       {gpu_score:.2f} / 25")
report_lines.append(f"Benchmark Score: {benchmark_score:.2f} / 30")
report_lines.append(f"⭐ TOTAL SCORE:   {total_score:.2f} / 100")
report_lines.append(f"🎯 SYSTEM TIER:   {tier}")

# --- Add Tier Mapping Table ---
report_lines.append("\n### SYSTEM TIER MAPPING ###")
report_lines.append("Score Range     | Tier")
report_lines.append("--------------- | --------------------------------------------")
report_lines.append("90–100          | 🚀 Extreme Workstation – Top 1% elite performance")
report_lines.append("80–89           | 🔥 Heavy ML/Gaming Machine – Very powerful")
report_lines.append("65–79           | ⚙️ Mid-Range Research Rig – Balanced, capable")
report_lines.append("50–64           | 🧪 Entry-Level ML Machine – Good for prototyping")
report_lines.append("30–49           | 🐢 Basic Productivity System – Limited ML use")
report_lines.append("< 30            | ❌ Low-Grade System – Not suitable for ML/DL")

# --- Save Report ---
with open("gpu_report.txt", "w") as f:
    f.write("\n".join(report_lines))

# --- Save Matrix Benchmark Plot ---
plt.figure(figsize=(10, 6))
plt.plot(sizes, times, marker='o')
plt.title("Matrix Multiplication Time vs Size")
plt.xlabel("Matrix Size (N x N)")
plt.ylabel("Time (seconds)")
plt.grid(True)
plt.savefig("gpu_benchmark.png")

# --- Save Score Radar Chart ---
labels = ['CPU', 'RAM', 'GPU', 'Benchmark']
scores = [cpu_score, ram_score, gpu_score, benchmark_score]
max_scores = [25, 20, 25, 30]
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
scores += scores[:1]
max_scores += max_scores[:1]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
ax.plot(angles, scores, 'o-', linewidth=2, label='System Score')
ax.fill(angles, scores, alpha=0.25)
ax.set_thetagrids(np.degrees(angles[:-1]), labels)
ax.set_title(f"System Performance Radar\n{tier}", fontsize=12)
ax.set_ylim(0, max(max_scores))
ax.grid(True)
plt.savefig("system_score_radar.png")

print("✅ Report saved to gpu_report.txt")
print("✅ Benchmark plot saved as gpu_benchmark.png")
print("✅ Score radar chart saved as system_score_radar.png")
